In [ ]:
from concurrent.futures import ProcessPoolExecutor

from dbconnect.dbconnector import DatabaseConnector
from timeseries.model import TimeSeriesModel

**Definir a função para processar um produto**

In [ ]:
def process_product(product_id):
    # Configurar o conector do banco de dados
    db_connector = DatabaseConnector(
        db_type="postgres",
        username="kubex_adm",
        password="abcdefg",
        dsn="192.168.100.15:5432/kubex_db"
    )

**Definir a consulta SQL para extrair dados históricos de estoque**

In [ ]:
    with open("data/CLI_X_PROD_CLI_SALE.sql.j2", "r") as file:
        query_template = file.read()

**    **Renderiza o template com o ID do produto**

In [ ]:
    query = query_template.format({"product_id": product_id})

    ts_model = TimeSeriesModel(
        db_connector=db_connector,
        query=query.replace("{ product_id }", str(product_id)),
        date_column="DTNEG",
        value_columns=["SALDO_HISTORICO"],
        is_csv=False,
        periods=30
    )

    try:

**        **Executar o modelo e gerar previsões**

In [ ]:
        ts_model.run()
        # forecast = ts_model.get_forecast()
    except Exception as e:
        print(f"Erro ao processar o produto {product_id}: {e}")
        return None
    
    print(f"Previsão para o produto {product_id} concluída.")
    # print(forecast)

**    **Salvar previsões**

In [ ]:
    output_file = f"output/products_stock/forecast_estoque_{product_id}.csv"
    ts_model.save_forecast(output_file)

**    **Plotar previsões**

In [ ]:
    # ts_model.plot_forecast()
    ts_model.save_plots(f"output/forecast_estoque_{product_id}.png")

** **Definir a função para obter IDs dos produtos**

In [ ]:
def get_product_ids() -> list:
    query_products = """
    SELECT 
        P.CODPROD
    FROM 
        TGFPRO P
    WHERE 
        P.USOPROD = 'R'
        AND P.ATIVO = 'S'
        AND P.CODPROD IN(1,2,3,4,5,6,7,8,9,10)
    """

    

** **Conectar ao banco de dados**

In [ ]:
    db_connector = DatabaseConnector(
        db_type="oracle",
        username="sankhya",
        password="abcdefg",
        dsn="127.0.0.1:1521/orcl"
    )
    db_connector.connect()

    try:

**        **Consultar a lista de produtos**

In [ ]:
        products_df = db_connector.execute_query(query_products)
    except Exception as e:
        print(f"Erro ao consultar produtos: {e}")
        return []

    # Fechar a conexão
    db_connector.close()

    # Lista de IDs dos produtos
    product_ids = products_df["CODPROD"].tolist()

    return product_ids

**Obter IDs dos produtos**

In [ ]:
product_ids = get_product_ids()

**Processar produtos em paralelo**

In [ ]:
with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(process_product, product_ids)

print("Processamento concluído.")

**Fim do script**